# Ejemplo de Visualización GeoMap con Bokeh

### 1) Importando los datos

In [3]:
#importamos librerías principales
import numpy as np
import pandas as pd

In [4]:
#abrimos el conjunto de datos
data = pd.read_csv("D:/AI_GAMES/earthquakes.csv")

In [5]:
#visualizamos los últimos eventos 
data.tail()

,Year,Mo,Dy,Hr,Mn,Sec,Tsu,Vol,Location Name,Latitude,...,Total Missing,Total Missing Description,Total Injuries,Total Injuries Description,Total Damage ($Mil),Total Damage Description,Total Houses Destroyed,Total Houses Destroyed Description,Total Houses Damaged,Total Houses Damaged Description
4419,2020.0,8.0,18.0,0.0,3.0,48.0,NaN,NaN,PHILIPPINES: MASBATE,12.021,...,NaN,NaN,50.0,1.0,0.62,1.0,141.0,3.0,851.0,3.0
4420,2020.0,10.0,19.0,20.0,54.0,39.0,5755.0,NaN,ALASKA,54.608,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4421,2020.0,10.0,30.0,11.0,51.0,27.0,5757.0,NaN,GREECE: SAMOS; TURKEY: IZMIR,37.918,...,NaN,NaN,1054.0,4.0,NaN,3.0,NaN,1.0,NaN,1.0
4422,2020.0,12.0,27.0,21.0,39.0,14.0,5760.0,NaN,CHILE: OFF COAST CENTRAL,-39.343,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4423,2020.0,12.0,29.0,11.0,19.0,54.0,NaN,NaN,BALKANS NW: CROATIA: PETRINJA,45.422,...,NaN,NaN,26.0,1.0,NaN,2.0,NaN,1.0,NaN,NaN


In [6]:
#tomamos los últimos 10 eventos para trabajar el mapa sobre ellos
data_sample = data[["Mag","Latitude", "Longitude"]].tail(10).reset_index(drop=True)

In [7]:
#muestra de los 10 últimos eventos | solo datos relevantes (pueden incluirse otros)
data_sample

,Mag,Latitude,Longitude
0,5.4,38.558,44.023
1,7.0,-7.843,147.766
2,7.8,55.030,-158.522
3,4.8,20.872,104.541
4,5.1,36.476,-81.094
5,6.6,12.021,124.123
6,7.6,54.608,-159.655
7,7.0,37.918,26.790
8,6.7,-39.343,-74.990
9,6.4,45.422,16.255


### 2) Transformación de Lat/Long a Mercator Coords.

Bokeh utiliza el [**Sistema de Coordenadas Universal Transversal Mercator**](https://es.wikipedia.org/wiki/Sistema_de_coordenadas_universal_transversal_de_Mercator). Por tanto, no podemos utilizar los valores de Lat/Long para la localización. Sin embargo, estos serán útiles para la referencia en la colocación de puntos. Por tanto, debemos computar el proceso de transformación de valores latitudinales y longitudinales a coordenadas Mercator.

In [8]:
# Función de transformación de valores de lat/long a Mercator
def x_coord(x, y):
    """
    Toma como parámetros de entrada los valores de Latitud(x) y Longitud (y).
    Devuelve una tupla con las coordenadas Mercator equivalentes.
    
    Estas deberán ser almacenadas como una tupla en el pd.DataFrame org.
    """
    #definición lat/long
    lat = x
    lon = y
    
    #mavor r_major de transformación
    r_major = 6378137.000
    
    #nueva "latitud" mercator
    x = r_major * np.radians(lon)
    
    #escala de transformación
    scale = x/lon
    
    #nueva "longitud" mercator
    y = 180.0/np.pi * np.log(np.tan(np.pi/4.0 + 
        lat * (np.pi/180.0)/2.0)) * scale
    
    #coordenadas finales (tuple)
    return (x, y)

In [9]:
#computamos coordenadas mercator utilizando Latitude & Longitude
data_sample["mercator"]= data_sample.apply(
    lambda x: x_coord(x.Latitude, x.Longitude), 
    axis=1
)

In [10]:
#separamos las coordenadas mercator en X e Y
data_sample[['mercator_x', 'mercator_y']] = data_sample['mercator'].apply(pd.Series)

In [11]:
#previsualización
data_sample

,Mag,Latitude,Longitude,mercator,mercator_x,mercator_y
0,5.4,38.558,44.023,"(4900617.943192283, 4658555.1782730725)",4.900618e+06,4.658555e+06
1,7.0,-7.843,147.766,"(16449235.876558863, -875818.208298777)",1.644924e+07,-8.758182e+05
2,7.8,55.030,-158.522,"(-17646588.319531318, 7367690.679611435)",-1.764659e+07,7.367691e+06
3,4.8,20.872,104.541,"(11637450.887019612, 2376622.48348553)",1.163745e+07,2.376622e+06
4,5.1,36.476,-81.094,"(-9027342.786389727, 4366317.463940572)",-9.027343e+06,4.366317e+06
5,6.6,12.021,124.123,"(13817309.155733498, 1348098.436685851)",1.381731e+07,1.348098e+06
6,7.6,54.608,-159.655,"(-17772713.30260009, 7286155.599177742)",-1.777271e+07,7.286156e+06
7,7.0,37.918,26.790,"(2982249.158351799, 4567848.430509404)",2.982249e+06,4.567848e+06
8,6.7,-39.343,-74.990,"(-8347848.614587584, -4770923.150547308)",-8.347849e+06,-4.770923e+06
9,6.4,45.422,16.255,"(1809498.322844662, 5688203.221090662)",1.809498e+06,5.688203e+06


### 3) Visualización Bokeh

In [12]:
#importamos librerías esenciales
from bokeh.palettes import PRGn, RdYlGn
from bokeh.transform import linear_cmap,factor_cmap
from bokeh.models import ColorBar, NumeralTickFormatter
from bokeh.tile_providers import CARTODBPOSITRON,get_provider, Vendors
from bokeh.plotting import figure, show, output_notebook, save, ColumnDataSource

In [13]:
#definimos proveedor de mapa base

chosentile = get_provider(CARTODBPOSITRON)

#para mapa de fondo Blanco y Negro, descomentar la sig. línea
chosentile = get_provider(Vendors.STAMEN_TONER) 

In [14]:
#definimos fuente de información (nuestro dataframe de trabajo)
source = ColumnDataSource(data=data_sample)

In [15]:
#definimos una paleta de colores cualesquiera
palette = RdYlGn[10] 

In [16]:
#definimos el rango de coloración del mapa con base a una variable (magnitud del sismo)
color_mapper = linear_cmap(
    field_name = 'Mag', #Magnitud del sismo
    palette = palette, low = data_sample['Mag'].min(), 
    high = data_sample['Mag'].max()
)

In [17]:
#definimos las variables de información de nuestro 'p' (plot)
p = figure(
    title = 'Last 10 Earthquakes in the World', 
    x_axis_type= "mercator", 
    y_axis_type= "mercator", 
    x_axis_label= "Longitude", 
    y_axis_label= "Latitude",
    plot_width=800, plot_height=400
)

In [18]:
#añadimos el mapa base seleccionado
p.add_tile(chosentile)

TileRenderer(id='1048', ...)

In [19]:
# Añadimos los puntos de las variables de información (Mag) según coords. mercator
p.circle(
    x = 'mercator_x', 
    y = 'mercator_y', 
    color = color_mapper, 
    source=source, 
    size=20, 
    fill_alpha = 0.7
)

GlyphRenderer(id='1055', ...)

In [20]:
#Definimos barra de color para identificar rango de valores de intensidad de Mag.
color_bar = ColorBar(
    color_mapper=color_mapper['transform'], 
    formatter = NumeralTickFormatter(format='0.0[0000]'), 
    label_standoff = 13, width=5, location=(0,0)
)

#posición de la barra (a la derecha del plot)
p.add_layout(color_bar, 'right')

In [21]:
#solicitamos el plot en el notebook
output_notebook()

Loading BokehJS ...

In [22]:
#plot final
show(p)

### 4) Acotaciones

Como pueden ver, el ploteo del mapa que pensamos no es tan complicado. Lo que sí resultará trabajoso serán dos cosas, a mi parecer.

    1. Computar la aparición/desaparición de los puntos en el mapa con base al tiempo y a la intensidad del sismo.
    2. Definir el tamaño/color adecuado para los puntos de rastro que permanecerán tras la posterior aparición del sismo.
    3. Elegir el mapa base (hay muchos mapas, desde este típico hasta el tradicional, en blanco y negro, por relieve, etc.).
    
El punto 2 es, junto con el primero, el más crucial. Si ustedes usan el total del conjunto de datos (es decir, usan el dataframe ```data``` en lugar de ```data_sample```), se darán cuenta que resulta extremadamente fácil sobrecargar de información al plot en los puntos donde confluyen los sismos (al punto que todo el mapa de mundo termina colmado de puntos). No es para menos, ya que son siento miles de miles de sismos registrados. 

Les adjunto las fuentes que me ayudaron con el plot para que también las puedan revisar, además del paper que ya compartimos previamente por WhatsApp.

### Fuentes:

1. https://docs.bokeh.org/en/latest/docs/user_guide/geo.html
2. https://automating-gis-processes.github.io/2017/lessons/L5/interactive-map-bokeh.html
3. https://towardsdatascience.com/a-complete-guide-to-an-interactive-geographical-map-using-python-f4c5197e23e0
4. https://towardsdatascience.com/exploring-and-visualizing-chicago-transit-data-using-pandas-and-bokeh-part-ii-intro-to-bokeh-5dca6c5ced10
5. https://docs.bokeh.org/en/latest/docs/reference/palettes.html
6. https://stackoverflow.com/questions/43955805/bokeh-map-plot-lat-long-to-x-and-y
    